In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np

import mne
from mne.preprocessing import (
    compute_proj_ecg,
    compute_proj_eog,
    create_ecg_epochs,
    create_eog_epochs,
)
%matplotlib qt

In [2]:
raw = mne.io.read_raw_fif('pre_filtered_data_raw.fif', preload=True)

Opening raw data file pre_filtered_data_raw.fif...
    Range : 25000 ... 1583999 =     50.000 ...  3167.998 secs
Ready.
Opening raw data file /home/ijekt/Documents/cs_cog_fat/pre_filtered_data_raw-1.fif...
    Range : 1584000 ... 2135999 =   3168.000 ...  4271.998 secs
Ready.
Reading 0 ... 2110999  =      0.000 ...  4221.998 secs...


In [4]:
#raw.compute_psd(fmax=200).plot(picks="data", exclude="bads")
#raw.plot(duration=5, n_channels=20, scalings='auto')

In [3]:
# compute SSP projectors
ecg_projs, events = compute_proj_ecg(raw, n_grad=1, n_mag=1, n_eeg=1, reject=None, ch_name='ECG')
eog_projs, _ = compute_proj_eog(raw, n_grad=1, n_mag=1, n_eeg=1, reject=None, ch_name='EOG002')

Including 0 SSP projectors from raw file
Running ECG SSP computation
Using channel ECG to identify heart beats.
Setting up band-pass filter from 5 - 35 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 5.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 4.75 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 35.25 Hz)
- Filter length: 5000 samples (10.000 s)

Number of ECG events detected : 4875 (average pulse 69 / min.)
Computing projector
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 35 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hamming window
- Lower passband edge: 1.00
- Lower tr

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 338 out of 338 | elapsed:   34.2s finished


Not setting metadata
4875 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 4875 events and 301 original time points ...
1 bad epochs dropped
Adding projection: planar--0.200-0.400-PCA-01
Adding projection: axial--0.200-0.400-PCA-01
Adding projection: eeg--0.200-0.400-PCA-01
Done.
Including 0 SSP projectors from raw file
Running EOG SSP computation
Using EOG channel: EOG002
EOG channel index for this subject is: [0]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Selecting channel EOG002 for blink detection
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition b

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 338 out of 338 | elapsed:   30.3s finished


Not setting metadata
197 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 197 events and 201 original time points ...
0 bad epochs dropped
Adding projection: planar--0.200-0.200-PCA-01
Adding projection: axial--0.200-0.200-PCA-01
Adding projection: eeg--0.200-0.200-PCA-01
Done.


In [16]:
for title in ("Without", "With"):
    if title == "With":
        raw.add_proj(eog_projs)
    with mne.viz.use_browser_backend("matplotlib"):
        fig = raw.plot(duration=5, n_channels=20, scalings='auto')
    fig.subplots_adjust(top=0.9)  # make room for title
    fig.suptitle("{} projectors".format(title), size="xx-large", weight="bold")

3 projection items deactivated
Channels marked as bad:
none
Channels marked as bad:
none


In [4]:
# Create a copy of the raw data before applying SSP for comparison
ssp_eog = raw.copy()

# add projs as Object to raw data
ssp_eog.add_proj(eog_projs)

# Apply SSP projectors to the copy
ssp_eog.apply_proj()

3 projection items deactivated
Created an SSP operator (subspace dimension = 3)
3 projection items activated
SSP projectors applied...


<Raw | pre_filtered_data_raw.fif, 343 x 2111000 (4222.0 s), ~5.40 GB, data loaded>

In [7]:
ssp_eog.save('SSP_eog_applied_raw.fif', overwrite=True)

Writing /home/ijekt/Documents/cs_cog_fat/SSP_eog_applied_raw.fif
Closing /home/ijekt/Documents/cs_cog_fat/SSP_eog_applied_raw.fif
Writing /home/ijekt/Documents/cs_cog_fat/SSP_eog_applied_raw-1.fif
Closing /home/ijekt/Documents/cs_cog_fat/SSP_eog_applied_raw-1.fif
[done]


In [8]:
# Create a copy of the raw data before applying SSP for comparison
ssp_ecg = raw.copy()

# add projs as Object to raw data
ssp_ecg.add_proj(ecg_projs)

# Apply SSP projectors to the copy
ssp_ecg.apply_proj()

3 projection items deactivated
Created an SSP operator (subspace dimension = 3)
3 projection items activated
SSP projectors applied...


<Raw | pre_filtered_data_raw.fif, 343 x 2111000 (4222.0 s), ~5.40 GB, data loaded>

In [9]:
ssp_ecg.save('SSP_ecg_applied_raw.fif', overwrite=True)

Writing /home/ijekt/Documents/cs_cog_fat/SSP_ecg_applied_raw.fif
Closing /home/ijekt/Documents/cs_cog_fat/SSP_ecg_applied_raw.fif
Writing /home/ijekt/Documents/cs_cog_fat/SSP_ecg_applied_raw-1.fif
Closing /home/ijekt/Documents/cs_cog_fat/SSP_ecg_applied_raw-1.fif
[done]


In [ ]:
# Plot raw with SSP
raw_ssp.plot(duration=5, n_channels=20, scalings='auto')

In [2]:
raw = mne.io.read_raw_fif('SSP_eog_raw.fif', preload=True)

Opening raw data file SSP_eog_raw.fif...
    Read a total of 3 projection items:
        EOG-planar--0.200-0.200-PCA-01 (1 x 204)  idle
        EOG-axial--0.200-0.200-PCA-01 (1 x 102)  idle
        EOG-eeg--0.200-0.200-PCA-01 (1 x 30)  idle
    Range : 25000 ... 1583999 =     50.000 ...  3167.998 secs
Ready.
Opening raw data file /home/ijekt/Documents/cs_cog_fat/SSP_eog_raw-1.fif...
    Read a total of 3 projection items:
        EOG-planar--0.200-0.200-PCA-01 (1 x 204)  idle
        EOG-axial--0.200-0.200-PCA-01 (1 x 102)  idle
        EOG-eeg--0.200-0.200-PCA-01 (1 x 30)  idle
    Range : 1584000 ... 2135999 =   3168.000 ...  4271.998 secs
Ready.
Reading 0 ... 2110999  =      0.000 ...  4221.998 secs...


In [4]:
# Plot og raw data for comparison
raw.plot(duration=5, n_channels=20, scalings='auto')

Channels marked as bad:
none
